# Inference on Real Astrophysical Candidates

## Role in the Project
This script applies the trained machine learning classifiers to real-world observational data. It serves as the bridge between the theoretical training phase and astrophysical application.

Its primary function is to determine the likelihood that specific neutron stars or gravitational wave merger remnants are composed of Hadronic matter or Quark matter. Since observational data contains significant uncertainties (error bars), this script does not output a single prediction but rather a **probability distribution** obtained via Monte Carlo sampling.

## Physics and Equations

### 1. Modeling Observational Uncertainty
Real astrophysical measurements of Mass ($M$), Radius ($R$), and Tidal Deformability ($\Lambda$) are not point values but probability distributions. In this analysis, these measurements are treated as independent Gaussian distributions:

$$
P(x_{obs} | x_{true}) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left( -\frac{(x_{obs} - x_{true})^2}{2\sigma^2} \right)
$$

where $x \in \{M, R, \Lambda\}$ and $\sigma$ represents the reported $1\sigma$ measurement uncertainty.

### 2. Physicality Constraints
Not all random samples drawn from these distributions represent physically valid stars. Constraints are applied to filter out unphysical realizations:

*   **Causality and Collapse:** Samples with extremely small radii ($R < 8$ km) are discarded, as they approach the Schwarzschild radius ($R_s = 2GM/c^2$) or violate causality constraints for stable matter.
*   **Mass Lower Bound:** Samples with $M < 0.1 M_{\odot}$ are discarded as they are below the minimum mass for neutron star formation.
*   **Tidal Validity:** For objects with tidal data, $\Lambda$ must be non-negative.

### 3. Bayesian Marginalization (Soft Voting)
The final classification is not determined by the "best fit" values but by integrating the model's prediction over the entire uncertainty region.

Let $f(\mathbf{x})$ be the classifier's output function, which returns the probability $P(\text{Quark}|\mathbf{x})$ for a specific configuration $\mathbf{x} = (M, R, \Lambda)$. The marginalized probability $\bar{P}$ for a candidate object is computed as the expectation value over the measurement distribution:

$$
\bar{P}(\text{Quark}) = \int f(\mathbf{x}) \cdot P(\mathbf{x}_{obs} | \mathbf{x}) \, d\mathbf{x} \approx \frac{1}{N} \sum_{i=1}^{N} f(\mathbf{x}_i)
$$

where $\mathbf{x}_i$ are the $N$ Monte Carlo samples drawn from the error distributions.

## Algorithm and Calculations

1.  **Catalog Definition**
    A list of astrophysical candidates is defined, including:
    *   **GW170817:** The first binary neutron star merger detected via gravitational waves (provides $M, R, \Lambda$).
    *   **PSR J0740+66:** A massive pulsar measured by NICER (provides $M, R$).
    *   **PSR J0030+04:** An isolated pulsar measured by NICER (provides $M, R$).
    *   **HESS J1731-347:** A surprisingly light compact object (provides $M, R$).
    *   **GW190814 (secondary):** A mystery object in the "mass gap" ($2.6 M_{\odot}$).

2.  **Monte Carlo Sampling**
    For each candidate:
    *   $5000$ synthetic realizations are generated by sampling from Gaussian distributions defined by the reported means and uncertainties.
    *   Unphysical samples are filtered out using the constraints described above.

3.  **Model Selection**
    The script dynamically selects the appropriate classifier based on the available data:
    *   **Model A (Full Physics):** Used if Tidal Deformability ($\Lambda$) is available (e.g., GW170817). Inputs: $[M, R, \log_{10}\Lambda]$.
    *   **Model Geo (Geometric):** Used if only Mass and Radius are available (e.g., Pulsars). Inputs: $[M, R]$.

4.  **Prediction and Averaging**
    *   The selected Random Forest model predicts the probability of being a Quark Star for every valid sample in the Monte Carlo set.
    *   The mean of these probabilities is calculated to produce the final `P(Quark)`.
    *   A verdict is assigned: "QUARK" if $\bar{P} > 0.5$, otherwise "HADRONIC".

## Inputs and Outputs

### Inputs
*   **Trained Models:** The dictionary containing the optimized classifiers (`Model A` and `Model Geo`).
*   **Candidate Data:** Hardcoded values for Mass, Radius, and Lambda (with errors) for selected objects.

### Outputs
*   **Inference Table:** A printed summary showing:
    *   Candidate Name.
    *   Mean Mass and Radius.
    *   Model used for inference.
    *   Calculated Probability of being a Quark Star ($P(\text{Quark})$).
    *   Final Classification Verdict.

## Connection to the Overall Workflow
This script is the **final application** step. After the models are trained on synthetic theoretical data (in `train_model.py`) and validated (in `audit_performance.py`), this script uses them to answer the core scientific question: "What is the nature of the compact objects we actually observe?"